# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-8.36,92,98,0.71,RU,1696297231
1,1,kirensk,57.7853,108.1119,-1.10,94,92,2.24,RU,1696297329
2,2,waitangi,-43.9535,-176.5597,8.23,93,100,7.15,NZ,1696297246
3,3,edinburgh of the seven seas,-37.0676,-12.3116,10.01,57,81,9.74,SH,1696297223
4,4,bayramaly,37.6185,62.1671,15.86,23,71,5.35,TM,1696297329


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points( "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp =21

#cities that fit criteria
cities_criteria_fit = city_data_df.loc[(city_data_df["Max Temp"] < max_temp) & 
                                       (city_data_df["Max Temp"] > min_temp) & 
                                       (city_data_df["Wind Speed"] < 4.5 ) &
                                       (city_data_df["Cloudiness"] == 0 )]


# Drop any rows with null values
cities_criteria_fit.dropna(how='any')
# Display sample data
cities_criteria_fit

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,cusseta,32.3054,-84.7727,24.81,75,0,2.47,US,1696297337
98,98,laguna,38.4210,-121.4238,26.16,30,0,2.06,US,1696297340
182,182,maceio,-9.6658,-35.7353,21.69,100,0,1.54,BR,1696297207
220,220,fort bragg,35.1390,-79.0060,21.08,78,0,2.57,US,1696297288
228,228,villa hayes,-25.0931,-57.5236,26.05,38,0,4.12,PY,1696297356
230,230,hamilton,39.1834,-84.5333,23.00,75,0,0.00,US,1696297332
233,233,el abadia,36.2695,1.6861,24.74,54,0,1.96,DZ,1696297357
293,293,zaranj,30.9667,61.8833,22.83,16,0,0.89,AF,1696297364
322,322,brookings,44.3114,-96.7984,25.21,50,0,1.34,US,1696297307
345,345,griffith,41.5284,-87.4237,24.47,66,0,0.00,US,1696297371


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_criteria_fit[["City", "Country", "Max Temp", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
75,cusseta,US,24.81,32.3054,-84.7727,75,
98,laguna,US,26.16,38.4210,-121.4238,30,
182,maceio,BR,21.69,-9.6658,-35.7353,100,
220,fort bragg,US,21.08,35.1390,-79.0060,78,
228,villa hayes,PY,26.05,-25.0931,-57.5236,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cusseta - nearest hotel: Victory Lodge
laguna - nearest hotel: Holiday Inn Express & Suites
maceio - nearest hotel: Maceió Hostel e Pousada
fort bragg - nearest hotel: Airborne Inn Lodging
villa hayes - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
el abadia - nearest hotel: فندق بحري
zaranj - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
griffith - nearest hotel: Quality Inn & Suites Hammond
west plains - nearest hotel: Super 8 by Wyndham West Plains
tripoli - nearest hotel: الاصيل
colonia - nearest hotel: Wasserturm Hotel Cologne
north branch - nearest hotel: AmericInn
benjamin constant - nearest hotel: No hotel found
mccomb - nearest hotel: Baymont by Wyndham McComb
malibu - nearest hotel: No hotel found


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
75,cusseta,US,24.81,32.3054,-84.7727,75,Victory Lodge
98,laguna,US,26.16,38.4210,-121.4238,30,Holiday Inn Express & Suites
182,maceio,BR,21.69,-9.6658,-35.7353,100,Maceió Hostel e Pousada
220,fort bragg,US,21.08,35.1390,-79.0060,78,Airborne Inn Lodging
228,villa hayes,PY,26.05,-25.0931,-57.5236,38,No hotel found
230,hamilton,US,23.00,39.1834,-84.5333,75,North Vista Manor
233,el abadia,DZ,24.74,36.2695,1.6861,54,فندق بحري
293,zaranj,AF,22.83,30.9667,61.8833,16,No hotel found
322,brookings,US,25.21,44.3114,-96.7984,50,Quality Inn
345,griffith,US,24.47,41.5284,-87.4237,66,Quality Inn & Suites Hammond


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points("Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)